In [1]:
import os
import json

from langchain_neo4j import Neo4jGraph
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [2]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Neo4jGraph(refresh_schema=False)

In [3]:
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Contract) REQUIRE c.file_id IS UNIQUE;")
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Party) REQUIRE c.name IS UNIQUE;")

[]

In [4]:
query = """UNWIND $data AS row
MERGE (c:Contract {file_id: row.file_id})
SET c.summary = row.summary,
    c.contract_type = row.contract_type,
    c.effective_date = date(row.effective_date),
    c.contract_scope = row.contract_scope,
    c.renewal_term = row.renewal_term,
    c.term = row.term,
    c.end_date = CASE WHEN row.end_date IS NOT NULL THEN date(row.end_date) ELSE NULL END,
    c.total_amount = row.total_amount,
    c.governing_law = row.governing_law.country

FOREACH (party IN row.parties |
    MERGE (p:Party {name: party.name})
    MERGE (l:Country {name: party.location.country})
    MERGE (p)-[:LOCATED_IN]->(l)
    MERGE (p)-[pr:PARTY_TO]->(c)
    SET pr.role = party.role
)"""

In [5]:
with open('contract_data.json', 'r') as file:
    results = json.load(file)

In [6]:
graph.query(query, {"data": results})

[]

In [7]:
import getpass

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("GOOGLE_KEY")

GOOGLE_KEY ········


In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
embeddings_output = await embeddings.aembed_documents([el["summary"] for el in results])

params = []
for embedding, contract in zip(embeddings_output,results):
    params.append({"file_id": contract["file_id"], "embedding": embedding})

with open("contract_embedding.json", "w") as json_file:
    json.dump(params, json_file, indent=4)

graph.query("""UNWIND $data AS row
MATCH (c:Contract {file_id:row.file_id})
CALL db.create.setNodeVectorProperty(c, 'embedding', row.embedding)""", 
            {"data": params})

[]

In [9]:
graph.query("CREATE VECTOR INDEX contractSummary IF NOT EXISTS FOR (c:Contract) ON c.embedding")

[]